In [212]:
import os
import pandas as pd
import re
import numpy as np
from collections import defaultdict

In [152]:
def get_unigrams(data_folder, max_training_index):
    training_data = pd.DataFrame()
    test_data = pd.DataFrame()
    sentiments = ['POS', 'NEG']
    for sent in sentiments:
        review_folder = f'{data_folder}/{sent}'
        for file in os.listdir(review_folder):
            new = pd.read_csv(os.path.join(review_folder, file), sep='\t', header=None, names=['word', 'pos'])
            new['sentiment'] = (1 if sent == sentiments[0] else -1)            
            if int(file[2:5]) <= max_training_index:
                training_data = training_data.append(new, sort=False)
    return training_data

In [234]:
unigram_training = get_unigrams('data-tagged', 899)
unigram_training.head()

,word,pos,sentiment
0,Though,IN,1
1,made,VBN,1
2,in,IN,1
3,Canada,NNP,1
4,",",",",1


In [149]:
print('The proportion of positive reviews:', (unigram_training['sentiment'] == 1).sum()/len(unigram_training))

The proportion of positive reviews: 0.5240155187221734


In [140]:
unigram_pos = unigram_training[unigram_training['sentiment'] > 0]['word'].value_counts()
unigram_neg = unigram_training[unigram_training['sentiment'] < 0]['word'].value_counts()
unigram_pos = unigram_pos[unigram_pos >= 4]
unigram_neg = unigram_neg[unigram_neg >= 4]

In [141]:
unigram_pos = unigram_pos/sum(unigram_pos)
unigram_neg = unigram_neg/sum(unigram_neg)

In [142]:
unigram_pos

,             0.056183
the           0.046551
.             0.040455
a             0.025588
and           0.024817
                ...   
Terrance      0.000006
transplant    0.000006
66            0.000006
Dr            0.000006
landscapes    0.000006
Name: word, Length: 12417, dtype: float64

In [143]:
unigram_neg

,            0.052071
the          0.043810
.            0.042055
a            0.025217
to           0.022301
               ...   
concludes    0.000007
codes        0.000007
sentenced    0.000007
Hamill       0.000007
utilize      0.000007
Name: word, Length: 11343, dtype: float64

In [ ]:
def find_bigrams(tokens, min_freq=7):
    bigrams = defaultdict(int)
    for i in range(1, len(tokens)-1):
        bigram = f'{tokens[i-1]} {tokens[i]}'
        bigrams[bigram] += 1
    return {key : bigrams[key] for key in bigrams if bigrams[key] >= min_freq}

In [235]:
def get_bigrams(review_folder, sentiment, max_training_index):
    content = ''
    review_folder = f'data/{sentiment}'
    for file in os.listdir(review_folder):
        if int(file[2:5]) <= max_training_index:
            f = open(os.path.join(review_folder, file), 'r')
            content += f.read()
    # split from punctuation and whitespace
    # then remove trailing whitespace
    tokens = np.asarray(list(filter(None,(map(str.strip, re.split('(\W)', content))))))
    return find_bigrams(tokens)

In [236]:
positive_bigrams = get_bigrams('data', 'POS', 899)
negative_bigrams = get_bigrams('data', 'NEG', 899)

In [237]:
positive_bigrams = positive_bigrams/len(positive_bigrams)
negative_bigrams = negative_bigrams/len(negative_bigrams)

TypeError: unsupported operand type(s) for /: 'dict' and 'int'